# Movie Recommendations :
Build a movie recommendation system
using collaborative filtering and machine
learning techniques in Python.


In [4]:
# Import necessary libraries
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [5]:
# Load MovieLens dataset
ratings_data = pd.read_csv('/content/ratings.csv')
movies_data = pd.read_csv('/content/movies.csv')

In [6]:
# Merge movie and rating data
movie_rating_data = pd.merge(ratings_data, movies_data, on='movieId')

In [7]:
# Create Surprise Reader and Dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(movie_rating_data[['userId', 'movieId', 'rating']], reader)

In [8]:
# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [9]:
# Build and train the collaborative filtering model (SVD)
model = SVD()
model.fit(trainset)

In [10]:
# Make predictions on the test set
predictions = model.test(testset)

In [11]:
# Evaluate the model's performance
accuracy.rmse(predictions)

RMSE: 0.8711


0.8710854117001419

In [12]:
# Recommend movies for a specific user
user_id = 1
user_movies = movie_rating_data[movie_rating_data['userId'] == user_id]['movieId'].tolist()

In [13]:
# Get movies that the user has not rated
unrated_movies = movies_data[~movies_data['movieId'].isin(user_movies)]

In [14]:
# Create a test set for the unrated movies
testset_user = [(user_id, movie_id, 0) for movie_id in unrated_movies['movieId']]

In [15]:
# Make predictions for unrated movies
user_recommendations = model.test(testset_user)

In [16]:
# Get the top N recommendations
N = 10
top_recommendations = sorted(user_recommendations, key=lambda x: x.est, reverse=True)[:N]

In [22]:
# Merge movie and rating data
movie_rating_data = pd.merge(ratings_data, movies_data, on='movieId')

# Take input from the user for genre
user_genre = input("Enter a movie genre (or 'exit' to quit): ").lower()

while user_genre != "exit":
    # Get movies rated by the user
    user_movies = movie_rating_data[movie_rating_data['userId'] == user_id]['movieId'].tolist()

    # Get unrated movies of the specified genre
    unrated_genre_movies = movies_data[
        (~movies_data['movieId'].isin(user_movies)) &
        (movies_data['genres'].str.lower().str.contains(user_genre))
    ]

    # Create a test set for unrated genre movies
    testset_user_genre = [(user_id, movie_id, 0) for movie_id in unrated_genre_movies['movieId']]

    # Make predictions for unrated genre movies
    user_genre_recommendations = model.test(testset_user_genre)

    # Get the top N recommendations
    top_genre_recommendations = sorted(user_genre_recommendations, key=lambda x: x.est, reverse=True)[:N]

    # Display top recommendations
    print(f"\nTop {N} Movie Recommendations for User {user_id} in the genre '{user_genre}':")
    for recommendation in top_genre_recommendations:
        movie_title = movies_data.loc[movies_data['movieId'] == recommendation.iid, 'title'].values[0]
        predicted_rating = recommendation.est
        print(f"{movie_title} - Predicted Rating: {predicted_rating}")

    # Take input from the user again
    user_genre = input("\nEnter a movie genre (or 'exit' to quit): ").lower()

print("Exiting the recommendation system.")


Enter a movie genre (or 'exit' to quit): comedy

Top 10 Movie Recommendations for User 1 in the genre 'comedy':
Christmas Story, A (1983) - Predicted Rating: 4.186968802045883
Fish Called Wanda, A (1988) - Predicted Rating: 4.144829984230765
American Graffiti (1973) - Predicted Rating: 4.133712879450346
Intouchables (2011) - Predicted Rating: 4.116187477279416
Big Lebowski, The (1998) - Predicted Rating: 4.090687698096025
Annie Hall (1977) - Predicted Rating: 4.085006205325951
Grand Day Out with Wallace and Gromit, A (1989) - Predicted Rating: 4.067796640427559
Dr. Horrible's Sing-Along Blog (2008) - Predicted Rating: 4.037168486457745
Raising Arizona (1987) - Predicted Rating: 4.029331496766115
Kick-Ass (2010) - Predicted Rating: 4.027391510748542

Enter a movie genre (or 'exit' to quit): exit
Exiting the recommendation system.
